## Descripción del Dataset y Metodología de Extracción

### 1. El Dataset Utilizado

El dataset seleccionado para este estudio es **CVEfixes (Versión 1.0.8)**. Este conjunto de datos constituye una colección automatizada y curada de vulnerabilidades y sus correspondientes correcciones extraídas de proyectos de software de código abierto.

La información base se obtiene de los registros CVE (Common Vulnerabilities and Exposures) de la Base de Datos Nacional de Vulnerabilidades (NVD), vinculando estos registros con los repositorios de código fuente (GitHub, GitLab, Bitbucket) para extraer el código antes y después del parche de seguridad. El dataset está estructurado como una base de datos relacional que permite el análisis a nivel de commit, archivo y método. Para este trabajo, se seleccionó el nivel de abstracción de **método (función)**.

### 2. Consultas y Procesamiento de Datos

Para la construcción del dataset final orientado al entrenamiento de modelos de Machine Learning, se ejecutó un proceso de extracción y limpieza dividido en dos etapas: extracción SQL y post-procesamiento en Python.

#### A. Extracción SQL

Se utilizó la siguiente consulta SQL sobre la base de datos SQLite para unificar la información dispersa en las tablas `method_change`, `file_change`, `fixes` y `cwe_classification`. Esta consulta selecciona las métricas de código, el código fuente y genera la etiqueta de clasificación (target), asegurando la eliminación de registros duplicados exactos desde la fuente.


In [ ]:
SELECT DISTINCT
    m.code,
    m.parameters,
    f.programming_language,
    m.nloc,
    m.complexity,
    m.token_count,
    m.top_nesting_level,
    CASE 
        WHEN m.before_change = 'False' THEN 'Safe'
        ELSE cc.cwe_id 
    END AS cwe_id
FROM method_change m
JOIN file_change f ON m.file_change_id = f.file_change_id
JOIN fixes fx ON f.hash = fx.hash
LEFT JOIN cwe_classification cc ON fx.cve_id = cc.cve_id;

#### B. Limpieza de Datos (Post-Procesamiento)

Posterior a la extracción, se aplicó un script de limpieza en Python para mitigar el ruido en las etiquetas ("Label Noise"). Este paso es crítico debido a la naturaleza de los commits de corrección, donde un cambio puede no afectar a todas las funciones extraídas, generando ambigüedad.

Las acciones de limpieza fueron:

1. **Eliminación de duplicados exactos:** Se utilizó `drop_duplicates` para remover filas idénticas remanentes.

2. **Eliminación de inconsistencias (Falsos Cambios):** Se identificaron y eliminaron los fragmentos de código que presentaban etiquetas contradictorias (el mismo código marcado simultáneamente como 'Safe' y con un 'CWE-ID'). Esto asegura que el modelo no reciba entradas idénticas con salidas diferentes.

### 3. Características del Dataset Resultante

El dataset final (`dataset_ml_ready.csv`) cuenta con las siguientes características:

* **code:** Código fuente de la función.
* **parameters:** Parámetros de la función para contexto de entrada.
* **programming_language:** Lenguaje de programación (ej. C, PHP, Java).
* **nloc:** Número de líneas de código.
* **complexity:** Complejidad ciclomática.
* **token_count:** Número total de tokens.
* **top_nesting_level:** Nivel máximo de anidamiento.
* **cwe_id (Target):** Etiqueta de clasificación ('Safe' o el identificador del CWE).

In [ ]:
import pandas as pd
import numpy as np

# Configuracion de archivos
INPUT_FILE = 'dataset_con_params.csv'
OUTPUT_FILE = 'dataset_ml_ready.csv'

print(f"Cargando {INPUT_FILE}...")
# Cargamos el CSV generado por SQLite
df = pd.read_csv(INPUT_FILE)

print(f"Filas originales: {len(df)}")

# Si una fila es IDENTICA a otra, se elimina.
df.drop_duplicates(inplace=True)
print(f"Filas tras eliminar duplicados exactos: {len(df)}")

print("Buscando codigos con etiquetas contradictorias")
conteo_etiquetas = df.groupby('code')['cwe_id'].nunique()
codigos_ambiguos = conteo_etiquetas[conteo_etiquetas > 1].index

num_ambiguos = len(codigos_ambiguos)
print(f"Se encontraron {num_ambiguos} fragmentos de codigo con etiquetas contradictorias.")

if num_ambiguos > 0:
    print("Eliminando conflictos...")
    # Filtramos: Nos quedamos solo con los codigos que NO esten en la lista de ambiguos
    df_limpio = df[~df['code'].isin(codigos_ambiguos)]
else:
    df_limpio = df

print(f"Filas finales limpias: {len(df_limpio)}")
print(f"Guardando en {OUTPUT_FILE}...")
df_limpio.to_csv(OUTPUT_FILE, index=False)

print("Proceso finalizado correctamente.")